# Analytic order of Sha

Exercise 1. Write a function which takes as input an elliptic curve over Q, and outputs the analytic order of sha.

In [19]:
#preliminaryfunctions:
#compute Omega
def period(E):
    factor =  1 + 1/2*(1+sgn(E.discriminant()))
    b2 = E.b2()
    b4 = E.b4()
    b6 = E.b6()
    f = 4*x^3 + b2*x^2+2*b4*x + b6
  #  print(f)
    roots = f.roots(ring=CC)
    real_roots = [p[0] for p in roots if p[0].is_real()]
    nonreal_roots = [p[0] for p in roots if not p[0].is_real()]
    if len(real_roots) == 3:
        real_roots.sort(reverse=true)
        difference = [real_roots[0]-real_roots[1],real_roots[0]-real_roots[2]]
        a0 = sqrt(CC(difference[0]))
        b0 = sqrt(CC(difference[1]))
        if abs(a0-b0)>abs(a0+b0):
                     b0 =-b0
        P = a0.agm(b0)
        return(CC(factor*pi/P))
    else:
        e3 = real_roots[0]
        diff = e3-nonreal_roots[0]
        z = sqrt(diff)
        if z.real()<0:
            z = -z
        P = z.agm(z.conjugate())    
   # e3 = roots[2][0]
   # e1 = roots[0][0]
   # e2 = roots[1][0]
   # ees1 = [e1,e2,e3]
   # ees2 = [e2,e1,e3]
   # ees3 = [e3,e1,e2]
   # listees = [ees1,ees2,ees3]
    #ees.sort(reverse = true)
    #print(ees)
   # periods = []
   # for ees in listees:
   #     difference = [ees[0]-ees[1],ees[0]-ees[2]]
   #     a0 = sqrt(CC(difference[0]))
   #     b0 = sqrt(CC(difference[1]))
   #     if abs(a0-b0)>abs(a0+b0):
    #                 b0 =-b0
    #    periods.append(a0.agm(b0))
   # factor =  1 + 1/2*(1+sgn(E.discriminant()))
 #   print(periods)
   # P = 1
   # for Ps in periods:
     #   if imag(Ps)==0:
     #       P = Ps
    return(CC(factor*pi/P))
E = EllipticCurve([0, 0, 1, -7, 36])
period(E)
def regul(E):
    h = E.height_function()
    def pairing(x,y):
        z = 1/2*(h(x+y)-h(x)-h(y))
        return z
    genos = E.gens()
    mat = matrix(CC, len(genos), lambda i , j : pairing(genos[i],genos[j]))
    return(mat.det())


In [20]:
def analytic_sha_order(E):
    leading_coeff = CC(sympow.analytic_rank(E)[1])
    torsion_cardinality = CC(E.torsion_subgroup().cardinality())
    omega = period(E)
    tamagawa_product = E.tamagawa_product()
    regulator = regul(E)
   # print('Coefficient: '+str(leading_coeff))
   # print('Torsion_Cardinality: '+str(torsion_cardinality))
    #print('Real Period: '+str(omega))
   # print('TamagawaProduct: '+str(tamagawa_product))
    #print('Regulator: '+str(regul(E)))
    Shan = (torsion_cardinality*torsion_cardinality *leading_coeff) / (omega*regulator*tamagawa_product) 
    return(Shan)
    # TODO
analytic_sha_order(E)

1.00000028651251

In [148]:
####TamagawaNumbers

def tamagawa_product_manual(E):
    a1 = E.a1()
    a2 = E.a2()
    a3 = E.a3()
    a4 = E.a4()
    a6 = E.a6()
    cond = E.discriminant()
    Factors = list(factor(cond))
    primes = [fac[0] for fac in Factors]
    product=1
    R.<x,y> = PolynomialRing(QQ,2,order="lex")
    f = x^3 + a2*x^2+a4*x+a6
    F = y^2 + a1*x*y +a3*y - f
    for p in primes:
        Fred = F.change_ring(GF(p))
        Fx = Fred.gradient()[0]
        Fy = Fred.gradient()[1]
        I = ideal([Fred,Fx,Fy])
        solutions = I.groebner_basis()
        solx = solutions[0].coefficients()[1]
        soly = solutions[1].coefficients()[1]
        #SingularPointMovedto X,Y = 0
        G = F(x = x - ZZ(solx), y= y -ZZ(soly))
        a1r = G.monomial_coefficient(x*y)
        a2r = G.monomial_coefficient(x*x)
        a3r = G.monomial_coefficient(y)
        a4r = G.monomial_coefficient(x)
        a6r = G.monomial_coefficient(G(x=0,y=0))
        b2r = a1r^2+4*a2r
        b8r = a1r*a1r*a6r-a1r*a3r*a4r+4*a2r*a6r+a2r*a3r*a3r-a4r*a4r
        b6r = a3r*a3r+4*a6r
        #step2inTateAlg
        if p.divides(b2r)==false:
            n = ZZ.valuation(p)(cond)
            Rp.<T> = PolynomialRing(GF(p))
            g = T*T + GF(p)(a1r)*T-GF(p)(a2r)
            if g.is_irreducible() == false:
                c = n
            elif ZZ.valuation(2)(n)==0:
                c = 1
            else:
                c = 2
        #step3
        elif (p*p).divides(a6r)==false:
            c = 1
        #step4
        elif (p*p*p).divides(b8r) ==false:
            c = 2
        #step5
        elif (p*p*p).divides(b6r) ==false:
            a31 = a3r/p
            a62 = a6r/(p*p)
            Rp.<T> = PolynomialRing(GF(p))
            g = T*T + GF(p)(a31)*T-GF(p)(a62)
            if g.is_irreducible() == false:
                c = 1
            else:
                c = 3
        #step6       
        
        product = c*product
        return(product)   
E = EllipticCurve('11a1')
tamagawa_product_manual(E)

1

Exercise 2. Run your function on several of the elliptic curves in the LMFDB using the API, and verify your result with the order given there.

In [45]:
import json
import requests
URL_TRUNK = ( 
    "https://www.lmfdb.org/api/ec_curvedata/?_format=json&rank={}&"
    #"_curvedata=Clabel,ainvs,rank,sha"
)
rank_4_url = URL_TRUNK.format(str(4))
my_data = requests.get(url=rank_4_url).json()["data"]
my_data
truncated_data = [data for data in my_data[:15]]
truncated_data
List = [[EllipticCurve(yolo['ainvs']),analytic_sha_order(EllipticCurve(yolo['ainvs'])),yolo['sha']] for yolo in truncated_data]
def shatest(L, tol):
    a = L[1]
    b = L[2]
    if a-b < tol:
        print('Valid for '+str(L[0]))
    else:
        print('False for '+str(L[0])+'difference is'+str(a-b))
for L in List :
    shatest(L,0.001)

Valid for Elliptic Curve defined by y^2 + x*y = x^3 - x^2 - 79*x + 289 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 + x^2 - 72*x + 210 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 - 7*x + 36 over Rational Field
Valid for Elliptic Curve defined by y^2 + x*y = x^3 - 202*x + 1089 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 + x^2 - 2*x + 42 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 + x^2 - 32*x + 72 over Rational Field
Valid for Elliptic Curve defined by y^2 + x*y = x^3 - 49*x + 120 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 - 67*x + 216 over Rational Field
Valid for Elliptic Curve defined by y^2 + x*y = x^3 - 25*x + 66 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 - 19*x + 60 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 - x^2 - 35*x + 72 over Rational Field
Valid for Elliptic Curve defined by y^2 + y = x^3 + x^2

Exercise 3. Can you extend the function to arbitrary number fields?

Interrupting PARI/GP interpreter...


KeyboardInterrupt: Ctrl-c pressed while running PARI/GP interpreter